**SPAM EMAIL CLASSIFICATION**

---


In [2]:
import os
import collections
import nltk
import urllib.request
import os
import random
from nltk.classify import NaiveBayesClassifier, accuracy
from sklearn.utils import shuffle
import concurrent.futures
import random
import re
import pickle
nltk.download('punkt')
# Importing all Required packages


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Define few stop words
stop_words = {
    'ourselves', 'hers', 'between', 'yourself', 'but', 'again','there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they',
    'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into','of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as',
    'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we','these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more',
    'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above','both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any',
    'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does','yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can',
    'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where','too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't',
    'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how','further', 'was', 'here', 'than'}
    

We will be using Euron spam dataset for spam email classification problem. The euron datasets ar present at the below location:
http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/index.html


The Euron dataset contains spam (malicious email) and ham (non-malicious email) two folders 
inside the big folder. Each folder spam and ham then again contain many text files. Let's load the data of the files to a list.

**Donwloading the dataset extract and loading it ETL**

In [4]:

# Define URLs of all six tar files
urls = [
    "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron1.tar.gz",
    "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron2.tar.gz",
    "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron3.tar.gz",
    "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron4.tar.gz",
    "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron5.tar.gz",
    "http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron6.tar.gz"
]
positive_samples = []
negative_samples = []

# Create a directory to store all the tar files
os.makedirs('enron', exist_ok=True)

# Download and extract each tar file in parallel
def download_extract(url):
    filename = os.path.join('enron', url.split("/")[-1])
    urllib.request.urlretrieve(url, filename)
    os.system(f"tar -xvf {filename} -C enron")
    os.remove(filename)  # Remove the tar file after extraction
    
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(download_extract, url) for url in urls]
    concurrent.futures.wait(futures)

# Define function to load files from a given directory
def load_files(directory):
    full_data = []
    for file_name in os.listdir(directory):
        with open(os.path.join(directory, file_name), 'r', encoding='ISO-8859-1') as f:
            full_data.append(f.read())
    return full_data

# Load positive samples (spam) and negative samples (ham) from all six directories
for i in range(1, 7):
    spam_dir = os.path.join('enron', f'enron{i}', 'spam')
    ham_dir = os.path.join('enron', f'enron{i}', 'ham')
    positive_samples += load_files(spam_dir)
    negative_samples += load_files(ham_dir)


**Preporcessing the Data**

In [5]:
# Preprocessing the data includes lemmatization, tokenization and stop word removal
nltk.download('wordnet')
def preprocess_sentence(sentence):
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w.lower()) for w in nltk.word_tokenize(sentence) if w.lower() not in stop_words]
    word_counts = collections.Counter(tokens)
    uncommon_words = word_counts.most_common()[:-10:-1]
    tokens = [w for w in tokens if w not in uncommon_words]
    return tokens
# Let us have a look at an email
for email in positive_samples[:1]:
  print(email)

# preprocess sentences 
positive_samples = [preprocess_sentence(email) for email in positive_samples]
negative_samples = [preprocess_sentence(email) for email in negative_samples]

# label samples
positive_samples = [(email, 1) for email in positive_samples]
negative_samples = [(email, 0) for email in negative_samples]
all_samples = positive_samples + negative_samples
random.shuffle(all_samples)
print(f"{len(all_samples)} emails processed")

[nltk_data] Downloading package wordnet to /root/nltk_data...


Subject: market watch undervalued report f
april 2004 top pick of the month
life energy and technology holdings , inc .
( otcbb : leth )
leth receives $ 250 , 000 , 000 in financing to fund the manufacture
of the environmentally friendly biosphere process system
waste - to - energy units in the united states .
first unit to roll - out in new orleans in early second quarter .
we are expecting earth - shattering upcoming news leading a strong
rally in leth for a company that has announced over $ 100 million
in sales orders in the past year , and tops that record - setting
achievement by acquiring the equivalent of $ 8 . 62 per share in
cash for major worldwide expansion .
* * our readers grabbed substantial profits for our march pick * *
ushg featured at . 75
reached 3 . 65 in 8 days !
traded as high as 7 . 00 since !
the biosphere process system - soaring worldwide demand :
leth is utilizing the unique proprietary technology of their
biosphere process system to generate revenue from the

In [6]:

# Feature extraction
def feature_extraction(tokens):
    # Each word will be a feature and feature value| will be word count
    return dict(collections.Counter(tokens))
    # features = [(feature_extraction(corpus), label) for corpus, label in all_samples]
features = [(feature_extraction(corpus), label)
              for corpus, label in all_samples]

In [ ]:
features[:3]

[({'subject': 1,
   ':': 1,
   'software': 1,
   'incredibly': 1,
   'low': 2,
   'price': 1,
   '(': 1,
   '83': 1,
   '%': 1,
   'lower': 1,
   ')': 1,
   '.': 14,
   'unconscionable': 1,
   'botching': 1,
   'gentle': 1,
   'always': 1,
   ',': 9,
   'vary': 1,
   'river': 1,
   'joy': 1,
   'hour': 1,
   'phrase': 1,
   'early': 2,
   'hair': 1,
   'grass': 1,
   'century': 1,
   'numeral': 1,
   'verb': 1,
   'wire': 1,
   'behind': 1,
   'thousand': 1,
   'face': 1,
   'common': 1,
   'fine': 1,
   'snow': 1,
   'long': 1,
   'lie': 1,
   'case': 1,
   'wash': 1,
   'job': 1,
   'complete': 1,
   'make': 1,
   'cover': 1,
   'bank': 1,
   'hit': 1,
   'soldier': 1,
   'serve': 1,
   'well': 1,
   'force': 1,
   'fly': 1,
   'every': 1,
   'round': 1,
   'laugh': 1,
   'smell': 1,
   'mind': 1,
   'rather': 1,
   'strong': 1,
   'village': 1,
   'lay': 1,
   'fraction': 1,
   'north': 1,
   'trade': 1,
   'sit': 1,
   'particular': 1},
  1),
 ({'subject': 3,
   ':': 13,
   'june':

**Training and Testing The Model**

In [7]:
# train test split
def train_test_split(dataset, train_size=0.8):
    num_train_samples = int(len(dataset) * train_size)
    return dataset[:num_train_samples], dataset[num_train_samples:]

training_set, test_set = train_test_split(features, train_size=0.7)
model = nltk.classify.NaiveBayesClassifier.train(training_set)
training_error = nltk.classify.accuracy(model, training_set)
print(f'Model training complete. Accuracy on training set: {training_error}')
testing_error = nltk.classify.accuracy(model, test_set)
print(f'Accuracy on test set: {testing_error}')

Model training complete. Accuracy on training set: 0.9952968094572264
Accuracy on test set: 0.9887296094908552


# **Custom Testing Of the Project**



In [13]:
def check_spam(input_data):
    preprocessed_email = preprocess_sentence(input_data)
    features = feature_extraction(preprocessed_email)
    prediction = model.classify(features)
    if prediction == 1:
        print("SPAM EMAIL")
    else:
        print("HAM EMAIL")


In [14]:

a = """
	
Hello, user-1123454



364 days ago, you registered on our platform for automatic cloud Bitcoin mining (collection) by living your device to our platform by IP address.

You were not active in your personal account, but the collection of cryptocurrency accrued automatically from your device.



You balance $24356.45



Your account will be deleted after 24 hour

Best regards support team Bit_Bonus
"""

custom_email2 = """
Greetings, I am Mrs.Fatim Adama aging widow of 62 years old suffering
from long time illness.I have some funds I inherited from my late
husband, the sum of ($18,500,000.00 Million Dollars) and I needed a
very honest and God fearing who can withdraw this money this funds use
it for Charity works.I found your email address from the internet after
honest prayers to the LORD to bring me a helper and i decided to
contact you if you may be willing and interested to handle these trust
funds in good faith. More detail will be giving after receiving your
reply.Please kindly respond quickly for further details through my
private e_mail address:(mrs.fatimadama@yahoo.com) Warmest Regards,
Mrs.Fatim Adama.
"""

test = """
Dear students,
Kindly find the updated schedule for the internship presentation. Submit the draft copy of the report on or before 10-4-2023.

Note: Change in schedule is not permitted.
 
Regards,

Pavithra D S
Assistant Professor
Dept of CSE
Canara Engineering College
Benjanapadavu
"""

check_spam(a)


Hi
SPAM EMAIL
